In [ ]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import matplotlib.pyplot as plt
from itertools import islice

from torchlensmaker.raytracing import super_refraction, clamped_refraction, ray_point_squared_distance, refraction
from torchlensmaker.optics import FocalPointLoss, ParallelBeamUniform, ParallelBeamRandom, FixedGap, RefractiveSurface, OpticalStack
from torchlensmaker.shapes import Parabola, PiecewiseLine, Line, CircularArc, BezierSpline
from torchlensmaker.training import render, optimize

In [ ]:

lens_width = 15.0
#surface1 = BezierSpline(width=lens_width, init=([3.0], [0.2*lens_width, 1.2*lens_width], [4.8]))
surface1 = BezierSpline(lens_width)
surface2 = BezierSpline.share(surface1, scale=-1)

# TODO enforce CX > X, i.e. control point within knots

optics = OpticalStack([
    ParallelBeamUniform(radius=0.99*lens_width),
    FixedGap(15.),
    
    RefractiveSurface(surface1, (1.0, 1.49)),
    FixedGap(15.),
    RefractiveSurface(surface2, (1.49, 1.0)),
    
    FixedGap(12.0),
    FocalPointLoss(),
])


render(optics, num_rays=10)

optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), 5e-2),
    num_rays = 20,
    num_iter = 150
)

render(optics, num_rays=10)
surface1.dump()

In [ ]:
from torchlensmaker.optics import Lens
from torchlensmaker.export3d import lens_to_part

lens = Lens(optics.stack[2], optics.stack[3], optics.stack[4])

part = lens_to_part(lens)
part